# Поиск токсичных комментариев

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные на основе набора данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Загрузка и подготовка

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
from tqdm import tqdm
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
RANDOM_STATE = 10
cv_param = 3

In [3]:
def about_df(list_df):
    '''выводит информацию о всех датасетах из списка list_df'''
    for c, i in enumerate(list_df):
        print("**** DATASET_"+str(c+1)+' ****')
        display(i.info())
        display(i.describe().round(2))
        print(f'Количество дубликатов: {i.duplicated().sum()}')
        print('Количество пропущенных значений:', i.isnull().sum(), sep='\n')
        print()

In [4]:
data = pd.read_csv('/datasets/toxic_comments.csv')
corpus = list(data['text'])

In [5]:
about_df([data])

**** DATASET_1 ****
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


None

,Unnamed: 0,toxic
count,159292.00,159292.0
mean,79725.70,0.1
std,46028.84,0.3
min,0.00,0.0
25%,39872.75,0.0
50%,79721.50,0.0
75%,119573.25,0.0
max,159450.00,1.0


Количество дубликатов: 0
Количество пропущенных значений:
Unnamed: 0    0
text          0
toxic         0
dtype: int64



In [6]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [7]:
# Unnamed: 0 - лишняя колонка, удалим
data = data.drop(['Unnamed: 0'], axis=1)

In [8]:
data.toxic.value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Как видно, токсичных комментариев намного меньше (16 тыс против 143). Обработаем входные тексты (очистим от лишних символов и лемматизируем):

In [9]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [10]:
nlp = spacy.load('en_core_web_sm')  # загрузим предварительную обученную модель для английского языка
stopwords = nlp.Defaults.stop_words  # импортируем стоп-слова

def lemmatize_text_spacy(text):
    '''Очистка от лишних символов и лемматизация'''
    
    cleared_text = nlp(" ".join(re.sub(r'[^a-zA-z]', ' ', text).split()))
    lemmatized_output = ' '.join([token.lemma_ for token in cleared_text])
    return lemmatized_output

In [11]:
tqdm.pandas()
data['lemm_text'] = data['text'].progress_apply(lemmatize_text_spacy)
data = data.drop(['text'], axis=1)

100%|██████████| 159292/159292 [40:52<00:00, 64.95it/s] 


In [12]:
data.head()

,toxic,lemm_text
0,0,explanation why the edit make under my usernam...
1,0,d aww he match this background colour I m seem...
2,0,hey man I m really not try to edit war it s ju...
3,0,More I can t make any real suggestion on impro...
4,0,you sir be my hero any chance you remember wha...


Разобъем данные на стратифицированные выборки:

In [13]:
X_train_1, X_test, y_train_1, y_test = train_test_split(data.drop(['toxic'], axis=1), 
                                                        data['toxic'], 
                                                        test_size=0.2, 
                                                        random_state=RANDOM_STATE,
                                                        stratify = data['toxic'])

X_train, X_valid, y_train, y_valid = train_test_split(X_train_1, 
                                                      y_train_1, 
                                                      test_size=0.25,
                                                      random_state=RANDOM_STATE,
                                                      stratify=y_train_1)

print('train size:', X_train.shape, ' valid size:', X_valid.shape, ' test size:', X_test.shape)

train size: (95574, 1)  valid size: (31859, 1)  test size: (31859, 1)


In [14]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [15]:
X_train = count_tf_idf.fit_transform(X_train['lemm_text'])
X_valid = count_tf_idf.transform(X_valid['lemm_text'])
X_test = count_tf_idf.transform(X_test['lemm_text'])

In [16]:
X_train

<95574x117339 sparse matrix of type '<class 'numpy.float64'>'
	with 2189802 stored elements in Compressed Sparse Row format>

**Вывод**
<br> Загруженные данные содержат 159 тыс. отзывов. Большинство из них положительные. Данные комментарии были очищены от лишних символов и лемматизированы для моделирования. Пропуски и дубликаты в данных отсутствуют.

## Обучение

Посмотрим, сколько очков даст простая регрессия на кросс-валидации (для сокращения времени параметр cv сделаем 3):

In [17]:
%%time
model = LogisticRegression()
f1_train = cross_val_score(model, 
                           X_train, 
                           y_train, 
                           cv=cv_param, 
                           scoring='f1').mean()
print('F1 на кросс-валидации', f1_train)

F1 на кросс-валидации 0.6881943299356603
CPU times: user 48.8 s, sys: 58.7 s, total: 1min 47s
Wall time: 1min 47s


Попробуем задать автоматическую настройку весов в обратной пропорции к частоте классов:

In [18]:
%%time
model = LogisticRegression(class_weight='balanced')
f1_train = cross_val_score(model, 
                           X_train, 
                           y_train, 
                           cv=cv_param, 
                           scoring='f1').mean()
print('F1 на кросс-валидации с балансировкой', f1_train)

F1 на кросс-валидации с балансировкой 0.7493776507638096
CPU times: user 36.9 s, sys: 43.6 s, total: 1min 20s
Wall time: 1min 20s


Виден прирост метрики, но все-равно это меньше целевых 0.75. Попоробуем перебрать разные параметры в модели:

In [19]:
%%time
grid_params = {'C': [10, 20, 30], 'penalty': ['l1', 'l2']}

model_lr = GridSearchCV(model, 
                        cv=cv_param,
                        param_grid=grid_params,
                        n_jobs=-1)

model_lr.fit(X_train, y_train)

CPU times: user 3min 25s, sys: 4min 6s, total: 7min 32s
Wall time: 7min 32s


GridSearchCV(cv=3, estimator=LogisticRegression(class_weight='balanced'),
             n_jobs=-1,
             param_grid={'C': [10, 20, 30], 'penalty': ['l1', 'l2']})

In [20]:
model_lr.best_estimator_

LogisticRegression(C=20, class_weight='balanced')

In [21]:
pred = model_lr.predict(X_valid)
f1_valid = f1_score(y_valid, pred)
f1_valid

0.7643349464730899

Посмотрим, как с задачей справится LGBMClassifier:

In [22]:
%%time
model_l = LGBMClassifier()
grid_params = {'n_estimators': [1000, 1500], 'max_depth': [5, 9]}

model_lgbm = GridSearchCV(model_l, 
                          cv=cv_param,
                          param_grid=grid_params,
                          n_jobs=-1,
                          verbose=10)

model_lgbm.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV 1/3; 1/4] START max_depth=5, n_estimators=1000..............................
[CV 1/3; 1/4] END ............max_depth=5, n_estimators=1000; total time= 4.8min
[CV 2/3; 1/4] START max_depth=5, n_estimators=1000..............................
[CV 2/3; 1/4] END ............max_depth=5, n_estimators=1000; total time= 4.2min
[CV 3/3; 1/4] START max_depth=5, n_estimators=1000..............................
[CV 3/3; 1/4] END ............max_depth=5, n_estimators=1000; total time= 4.7min
[CV 1/3; 2/4] START max_depth=5, n_estimators=1500..............................
[CV 1/3; 2/4] END ............max_depth=5, n_estimators=1500; total time= 4.4min
[CV 2/3; 2/4] START max_depth=5, n_estimators=1500..............................
[CV 2/3; 2/4] END ............max_depth=5, n_estimators=1500; total time= 4.0min
[CV 3/3; 2/4] START max_depth=5, n_estimators=1500..............................
[CV 3/3; 2/4] END ............max_depth=5, n_esti

GridSearchCV(cv=3, estimator=LGBMClassifier(), n_jobs=-1,
             param_grid={'max_depth': [5, 9], 'n_estimators': [1000, 1500]},
             verbose=10)

In [23]:
model_lgbm.best_estimator_

LGBMClassifier(max_depth=9, n_estimators=1500)

In [24]:
pred = model_lgbm.predict(X_valid)
f1_valid = f1_score(y_valid, pred)
f1_valid

0.7679073033707866

С параметром С = 50 логистическая регрессия укладывается в необходимую границу f1. Но LGBMClassifier с параметрами: max_depth=9 и n_estimators=1500 оказался немного получше (f1 на валидационной выборке 0.768) при этом сильно медленнее. Ввиду высокой вычислительной сложности задачи (даже логистической регресси потребовалось больше минуты времени) и удовлетворительного значения f1, проверять другие модели не станем. Посмотрим f1 на тестовой выборке:

In [25]:
pred = model_lgbm.predict(X_test)
f1_test = f1_score(y_test, pred)
f1_test

0.7772593744599965

## Выводы

Загруженные данные содержат 159 тыс. отзывов. Большинство из них положительные. Данные комментарии были очищены от лишних символов и лемматизированы для моделирования. Пропуски и дубликаты в данных отсутствуют.
<br>Для определения негативных / позитивных комментариев были использованы логистическая регрессия и LGBMClassifier. Лучше оказалась LGBMClassifier с параметрами: max_depth=9 и n_estimators=1500. Метрика F1 на тестовой выборке для нее составила 0.777. Ввиду хорошего результата по данной модели, прочие модели не рассматривались в работе с целью сокращения времени обучения (даже логистической регрессии потребовалось более минуты времени для обучения).